Reference:
- https://www.youtube.com/watch?v=KfnhNlD8WZI
- https://www.youtube.com/watch?v=7n1SpeudvAE
- https://www.dlology.com/blog/how-to-convert-trained-keras-model-to-tensorflow-and-make-prediction/

In [ ]:
import pandas as pd
import numpy as np
import os

# Scikit-Learn
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.externals import joblib

# Tensorflow
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.python.platform import gfile
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense

In [ ]:
OUT_PATH_FILE = 'resources/output/'

In [ ]:
def load_data():
    dataset = datasets.load_iris()
    df = pd.DataFrame(dataset['data'], columns=dataset['feature_names'])
    df['target'] = dataset['target']
    df['target name'] = df['target'].map({i: x for i,x in enumerate(dataset['target_names'])})
    
    return df

In [ ]:
# Data loading
df = load_data()

df.shape

# Train Model

In [ ]:
def feature_target_split(df):
    X = df[[x for x in df.columns if 'target' not in x]].copy()
    y = df['target'].copy()
    
    return X, y

In [ ]:
# Feature-target separation
X, y = feature_target_split(df)

X.shape, y.shape

In [ ]:
y.value_counts()

In [ ]:
def encode_target(y):
    return pd.get_dummies(y)

def decode_target(y):
    return np.argmax(y, axis=1)

In [ ]:
y = encode_target(y)

y.shape

In [ ]:
def train_test(X, y, test_size=.2):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=0, stratify=y)

    X_train.reset_index(drop=True, inplace=True)
    X_test.reset_index(drop=True, inplace=True)
    y_train.reset_index(drop=True, inplace=True)
    y_test.reset_index(drop=True, inplace=True)
    
    return X_train, X_test, y_train, y_test

In [ ]:
# Split training & testing dataset
X_train, X_test, y_train, y_test = train_test(X, y)

X_train.shape, X_test.shape

In [ ]:
# Pre-processing pipeline
steps    = [
    ('std_scaler', StandardScaler()),
    ('mm_scaler', MinMaxScaler())
]
pipeline = Pipeline(steps, verbose=True)
X_train  = pipeline.fit_transform(X_train)

X_train.shape

In [ ]:
# Neural Network
model = Sequential()
model.add(Dense(32, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

model.fit(X_train, y_train, epochs=100)

In [ ]:
# Apply transformation to test dataset
X_test = pipeline.transform(X_test)

X_test.shape

In [ ]:
# Prediction on test dataset
y_pred = model.predict(X_test)
y_pred = decode_target(y_pred)
y_test = decode_target(y_test.values)

y_pred.shape, y_test.shape

In [ ]:
def classif_eval(y_true, y_pred):
    cofmat_df = pd.DataFrame(confusion_matrix(y_true, y_pred))
    cofmat_df.index.name   = 'True'
    cofmat_df.columns.name = 'Pred'

    print(cofmat_df)
    print()
    print(classification_report(y_true, y_pred, digits=5))

In [ ]:
# Model evaluation
classif_eval(y_test, y_pred)

In [ ]:
os.makedirs(OUT_PATH_FILE, exist_ok=True)

# Save scikit-learn models
for key, value in pipeline.named_steps.items():
    joblib.dump(value, f'{OUT_PATH_FILE}{key}.joblib')

# Save tensorflow keras model
model.save(f'{OUT_PATH_FILE}classif_model.H5')

# Restore Model

In [ ]:
# Feature-target separation
X, y = feature_target_split(df)

X.shape, y.shape

In [ ]:
# Split training & testing dataset
X_train, X_test, y_train, y_test = train_test(X, y)

X_train.shape, X_test.shape

In [ ]:
# Restore scikit-learn models
std_scaler = joblib.load(f'{OUT_PATH_FILE}std_scaler.joblib')
mm_scaler  = joblib.load(f'{OUT_PATH_FILE}mm_scaler.joblib')

In [ ]:
# Pre-processing pipeline
steps    = [
    ('std_scaler', std_scaler),
    ('mm_scaler', mm_scaler)
]
pipeline = Pipeline(steps, verbose=True)
X_test   = pipeline.transform(X_test)

# Freeze Model

In [ ]:
# This line must be executed before loading keras model.
K.set_learning_phase(0)

In [ ]:
# Restore models
model = load_model(f'{OUT_PATH_FILE}classif_model.H5')

model.inputs, model.outputs

In [ ]:
sess = K.get_session()
graph_def = sess.graph.as_graph_def()

In [ ]:
# Reference: https://github.com/Tony607/keras-tf-pb/blob/master/keras_freeze_tf_pb.ipynb
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    import tensorflow as tf
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def, output_names, freeze_var_names)
        return frozen_graph

In [ ]:
# Generate serialized GraphDef ProtoBuf
frozen_graph = freeze_session(K.get_session(), output_names=[x.op.name for x in model.outputs])

In [ ]:
# Save GraphDef ProtoBuf as single binary .pb file
tf.train.write_graph(frozen_graph, logdir=OUT_PATH_FILE, name='classif_model_TF.PB', as_text=False)

In [ ]:
# Load GraphDef ProtoBuf in new session
sess = tf.InteractiveSession()

f = gfile.FastGFile(f'{OUT_PATH_FILE}classif_model_TF.PB', 'rb')
graph_def = tf.GraphDef()

# Parses a serialized binary message into the current message
graph_def.ParseFromString(f.read())
f.close()

In [ ]:
sess.graph.as_default()

# Import a serialized tensorflow GraphDef protocol buffer and place into the current default Graph
tf.import_graph_def(graph_def)

In [ ]:
# Input & Output tensor
input_tensor  = sess.graph.get_tensor_by_name(f'import/{model.inputs[0].name}')
output_tensor = sess.graph.get_tensor_by_name(f'import/{model.outputs[0].name}')

input_tensor, output_tensor

In [ ]:
# Prediction (Single input & output)
y_pred = sess.run(output_tensor, {input_tensor: X_test})
y_pred = decode_target(y_pred)

y_pred.shape

# Example:
# Prediction (Multiple inputs & outputs)
# y_pred1, y_pred2 = sess.run([output_tensor1, output_tensor2], {
#     input_tensor1: X_test1,
#     input_tensor2: X_test2,
# })

In [ ]:
# Model evaluation
classif_eval(y_test, y_pred)